In [1]:
import sys

sys.path.insert(0, '../03_Remake_with_TDD/algorithms')
from ht_time import *

sys.path.insert(0, '../03_Remake_with_TDD/postgresql')
from ht_match_postgresql import *

In [2]:
ht_time = HattrickTime()
folder = ht_time.findLastSunOrWed()
# folder = '2018/04/08'
print("folder =", folder)

folder = 2018/06/13


In [3]:
import re
from bs4 import BeautifulSoup

In [4]:
import sys

sys.path.insert(0, '../97_Parse/')
from parseMatch import *

sys.path.insert(0, '../98_Crawler/01_DualMonitor/')
from DualMonitor import getIsDualMonitor

sys.path.insert(0, '../98_Crawler/02_WebBrowser/')
from FirefoxTab import clickTabOfFireFox
from crawMatch import *

sys.path.insert(0, '../98_Crawler/03_OnScreen/')
from Empty import clickEmpty
from MoveScroll import moveScroll
from MoveFindMoveClick import moveByImage, moveByImageWithModify

sys.path.insert(0, '../98_Crawler/04_Login/')
from HattrickLogInOut import clickHattrickLogin, clickHattrickLogout

In [34]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

clickTabOfFireFox(4, isDualMonitor)

clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

# clickHattrickLogout(1270, 370, isDualMonitor)

use 2 monitor = 3840 x 1080 isDualMonitor= True


In [5]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick HeavyDefence Tab
clickTabOfFireFox(4, isDualMonitor)

# # Hattrick Login by Facebook
clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

# Hattrick MyClub
moveScroll(20)
moveByImage(530, 342, 77, 42, isDualMonitor, 'screen-MyClub.png')

# Hattrick Match
clickEmpty(700, isDualMonitor)
moveScroll(20)
moveByImage(384, 698, 80, 108, isDualMonitor, 'screen-Match.png')

# Hattrick LastMatch
clickEmpty(831, isDualMonitor)
moveScroll(20)
# moveByImageWithModify(1040, 831, 15, -15, 58, 48, isDualMonitor, 'screen-LastMatch.png')
moveAndClick(1082, 841, isDualMonitor)
time.sleep(5)

# FirefoxFile
moveAndClick(90, 15, isDualMonitor)

# SavePageAs
time.sleep(3)
moveAndClick(91, 130, isDualMonitor)

# Type match
time.sleep(3)
typeFileName()

# rwoo folder
moveAndClick(570, 460, isDualMonitor)

# 02_workspace folder
moveAndClicks(720, 440, isDualMonitor, 2)

# 04_Hattrick folder
moveAndClicks(720, 500, isDualMonitor, 2)

# HattrickStudy
moveAndClicks(720, 415, isDualMonitor, 2)

# 01_Heavy2Defence
moveAndClicks(720, 440, isDualMonitor, 2)

# 2018
moveAndClicks(720, 440, isDualMonitor, 2)

# Last Month
clickLastMonthOrDay()

# Last Day
clickLastMonthOrDay()

# Save
moveByImage(1280, 847, 91, 41, isDualMonitor, 'screen-Save.png')

# Hattrick Logout by Facebook
clickEmpty(336, isDualMonitor)
moveScroll(20)
clickHattrickLogout(1270, 370, isDualMonitor)

# Jupyter Tab
clickTabOfFireFox(7, isDualMonitor)

use 2 monitor = 3840 x 1080 isDualMonitor= True
screen-MyClub.png location on Screen ( 530 342 77 42 )
screen-Match.png location on Screen ( 384 698 80 108 )
screen-Save.png location on Screen ( 1280 847 91 41 )


In [6]:
from ht_file import *

In [7]:
HattrickFile = HattrickFile()
HattrickFile.check_and_delete_folder(folder + '/match_files')

2018/06/13/match_files		 : is exitsted
2018/06/13/match_files		 : will be removed
2018/06/13/match_files		 : is not exitsed


In [8]:
from ht_match import *

In [9]:
HattrickMatch = HattrickMatch()
match_dict_list = HattrickMatch.findMatchList(folder + '/match.html', 'mydatabase2')
match_str_list = HattrickMatch.matchDictListToMatchStrList(match_dict_list)

print("\ncat match_str_List\n")
for str_line in match_str_list:
    print(str_line)

Home

cat match_str_List

po,num,rt,sMin,eMin
KP,2,3.0,0,90
WB,54,3.0,0,90
WB,58,3.5,0,90
CD,33,4.5,0,90
CD,43,5.0,0,90
CD,59,4.0,0,90
W,63,6.5,0,90
W,15,6.5,0,80
W,7,5.0,80,90
IM,31,7.0,0,90
IM,10,5.5,0,70
IM,11,5.0,70,90
FW,16,6.0,0,80
FW,17,3.5,80,90


In [10]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

2018/06/13/match.txt	 : will be removed
2018/06/13/match.txt	 : is not existed


In [11]:
for line in match_str_list:
    with open (folder + '/match.txt', 'a') as f:
        f.write(line + "\n")

In [ ]:
# From match.txt to match table

In [12]:
import psycopg2

In [13]:
conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")

In [ ]:
# def dropMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """DROP TABLE IF EXISTS match ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
#dropMatchTable()

In [ ]:
# def createMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """
# CREATE TABLE match (
#     Date DATE,          --01
#     Po   VARCHAR(5),    --02 Position
#     Num  INT,           --03 Number
#     RT   NUMERIC(2, 1), --04 Rating
#     sMin INT,           --05 Start Min
#     eMin INT            --06 End Min
# ) ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
# createMatchTable()

In [14]:
def selectCountPlayer(conn):
    try:
        cur = conn.cursor()
        sql = "SELECT COUNT(*) FROM match" 
        
        cur.execute(sql)
        row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print(sql)
    return row[0]

In [15]:
selectCountPlayer(conn)

473

In [16]:
def insertMatchTable(time, row, conn):
    try:
        cur = conn.cursor()
        sql = """
INSERT INTO match ( 
    SELECT 
        to_date(%s, 'YYYY-MM-DD'), --01 
        %s, %s, %s, %s, %s         --02 03 04 05 06
) ;"""
        cur.execute(sql, ( time,
                           row[0], row[1], row[2], row[3], row[4] ) )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)
        print(row)

In [17]:
import csv
with open(folder + '/match.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match( '^[0-9]{1,2}', row[1] ) ) :
            insertMatchTable( folder.replace('/','-'), row, conn)

In [18]:
selectCountPlayer(conn)

487

In [19]:
def updateRatingOfMatchTable(conn):
    try:
        cur = conn.cursor()
        sql = """
update match 
    set rt = player.rt 
from player   
where 
    match.date = player.date 
    and player.date = player.last 
    and match.po = player.po 
    and match.num = player.num 
    and match.rt != player.rt
; """
        cur.execute(sql)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)

In [20]:
updateRatingOfMatchTable(conn)

In [21]:
def getTupleListFromMatch(query, conn):
    outputList = list();
    try:
        cur = conn.cursor()
        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    return outputList

In [22]:
query = ""
query += "\n" + "SELECT "
query += "\n" + "    po, num, rt :: VARCHAR(5), smin, emin "
query += "\n" + "FROM match "
query += "\n" + "WHERE "
query += "\n" + "    date ='" + folder.replace('/','-') + "'"
query += "\n" + "ORDER BY "
query += "\n" + "    CASE "
query += "\n" + "        WHEN po = 'KP' THEN 1"
query += "\n" + "        WHEN po = 'WB' THEN 2"
query += "\n" + "        WHEN po = 'CD' THEN 3"
query += "\n" + "        WHEN po = 'W'  THEN 4"
query += "\n" + "        WHEN po = 'IM' THEN 5"
query += "\n" + "        WHEN po = 'FW' THEN 6"
query += "\n" + "        ELSE 7 "
query += "\n" + "    END, "
query += "\n" + "    emin asc, smin desc"

tupleList = getTupleListFromMatch(query, conn)
for tuple in tupleList:
    print(tuple[0], tuple[1], tuple[2], tuple[3], tuple[4], sep='\t')

KP	2	3.0	0	90
KP	2	3.0	0	90
WB	54	2.5	0	90
WB	58	3.0	0	90
WB	54	2.5	0	90
WB	58	3.0	0	90
CD	33	4.5	0	90
CD	43	4.5	0	90
CD	59	3.5	0	90
CD	33	4.5	0	90
CD	43	4.5	0	90
CD	59	3.5	0	90
W	15	6.5	0	80
W	15	6.5	0	80
W	7	5.0	80	90
W	7	5.0	80	90
W	63	6.0	0	90
W	63	6.0	0	90
IM	10	5.5	0	70
IM	10	5.5	0	70
IM	11	5.0	70	90
IM	11	5.0	70	90
IM	31	6.5	0	90
IM	31	6.5	0	90
FW	16	5.5	0	80
FW	16	5.5	0	80
FW	17	3.5	80	90
FW	17	3.5	80	90


In [23]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

2018/06/13/match.txt	 : will be removed
2018/06/13/match.txt	 : is not existed


In [24]:
with open ( folder + '/match.txt', 'a' ) as f:
    f.write("po,num,rt,sMin,eMin\n")
    f.write( '\n'.join('%s,%s,%s,%s,%s' % myTuple for myTuple in tupleList) + '\n' )

In [25]:
hattrickMatchPostgresql = HattrickMatchPostgreSQL()

In [26]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'KP', [1,2])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [1,2])

date num min/rt |    01   |    02   |
----------------|---------|---------|
2018-05-20      |  90/4.0 |         |
2018-05-23      |         |  90/3.5 |
2018-05-27      |  90/4.0 |         |
2018-05-30      |         |  90/3.5 |
2018-06-03      |  90/4.0 |         |
2018-06-06      |         |  90/3.0 |
2018-06-10      |  90/4.0 |         |
2018-06-13      |         |  90/3.0 |


In [27]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'WB', [44,46,54,58])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [44,46,54,58])

date num min/rt |    44   |    46   |    54   |    58   |
----------------|---------|---------|---------|---------|
2018-05-20      |  90/3.5 |  90/4.5 |         |         |
2018-05-23      |         |         |  90/2.5 |  90/2.5 |
2018-05-27      |  90/3.5 |  90/4.5 |         |         |
2018-05-30      |         |         |  90/2.5 |  90/2.5 |
2018-06-03      |  90/3.5 |  90/4.5 |         |         |
2018-06-06      |         |         |  90/2.5 |  90/3.0 |
2018-06-10      |  90/3.5 |  90/4.5 |         |         |
2018-06-13      |         |         |  90/2.5 |  90/3.0 |


In [28]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'CD', [33,43,59,61,64,66])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [33,43,59,61,64,66])

date num min/rt |    33   |    43   |    59   |    61   |    64   |    66   |
----------------|---------|---------|---------|---------|---------|---------|
2018-05-20      |         |         |  90/3.0 |  90/2.5 |         |         |
2018-05-23      |  90/4.5 |  90/4.5 |         |         |  90/3.5 |         |
2018-05-27      |         |         |  90/3.0 |  90/3.0 |         |         |
2018-05-30      |  90/4.5 |  90/4.5 |         |         |  90/3.5 |         |
2018-06-03      |         |         |  90/3.0 |  82/3.0 |  83/3.5 |         |
2018-06-06      |  90/5.0 |  90/4.5 |         |         |         |         |
2018-06-10      |         |         |         |  90/3.0 |  90/3.5 |  59/2.5 |
2018-06-13      |  90/4.5 |  90/4.5 |  90/3.5 |         |         |         |


In [29]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'W', [7,13,15,48,63])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [7,13,15,48,63])

date num min/rt |    07   |    13   |    15   |    48   |    63   |
----------------|---------|---------|---------|---------|---------|
2018-05-20      |  10/5.0 |  80/6.5 |  90/7.0 |         |         |
2018-05-23      |  44/4.5 |         |         |         |  90/4.5 |
2018-05-27      |  83/4.5 |   7/6.5 |  90/7.0 |         |         |
2018-05-30      |         |         |  44/6.5 |         |  90/5.5 |
2018-06-03      |   8/5.5 |  80/6.5 |  90/7.0 |         |         |
2018-06-06      |         |         |  30/6.5 |         |  90/6.0 |
2018-06-10      |         |  80/6.0 |  90/7.0 |  10/4.0 |         |
2018-06-13      |  10/5.0 |         |  80/6.5 |         |  90/6.0 |


In [30]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'IM', [9,10,11,12,31])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [9,10,11,12,31])

date num min/rt |    09   |    10   |    11   |    12   |    31   |
----------------|---------|---------|---------|---------|---------|
2018-05-20      |  25/5.0 |         |  65/5.0 |         |  90/6.5 |
2018-05-23      |  30/5.0 |  75/5.0 |         |  60/4.5 |         |
2018-05-27      |  30/5.5 |         |  60/5.0 |         |  90/6.5 |
2018-05-30      |  60/5.5 |         |  30/5.0 |  90/4.0 |         |
2018-06-03      |  35/5.5 |         |  90/4.5 |  55/4.5 |         |
2018-06-06      |         |  20/5.5 |         |  90/4.5 |  70/7.0 |
2018-06-10      |  90/5.5 |         |  50/5.0 |  40/5.0 |         |
2018-06-13      |         |  70/5.5 |  20/5.0 |         |  90/6.5 |


In [31]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'FW', [16,17,18,60])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [16,17,18,60])

date num min/rt |    16   |    17   |    18   |    60   |
----------------|---------|---------|---------|---------|
2018-05-20      |         |         |  65/5.5 |  25/5.0 |
2018-05-23      |  75/5.5 |  15/4.5 |         |         |
2018-05-27      |         |         |  65/5.0 |  25/5.0 |
2018-05-30      |  80/5.0 |  10/4.0 |         |         |
2018-06-03      |         |         |  65/5.0 |  25/5.0 |
2018-06-06      |  80/5.5 |  10/4.0 |         |         |
2018-06-10      |         |         |  60/5.0 |  30/4.5 |
2018-06-13      |  80/5.5 |  10/3.5 |         |         |


In [32]:
conn.close()
print(conn)

<connection object at 0x7fda82b8f3d8; dsn: 'user=myuser password=xxx dbname=mydatabase2 host=localhost port=65432', closed: 1>


In [33]:
conn = None
print(conn)

None
